<a href="https://colab.research.google.com/github/sungkabkim/python_programming/blob/main/13%EC%A3%BC%EC%B0%A82_%EA%B9%80%EC%84%B1%EA%B0%91_24%EB%B0%95%EC%82%AC_2024285934.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### (1) 도서의 크기 정보 추출

In [ ]:
# 1. 검색 결과 페이지 가져오기

import gdown

gdown.download('https://bit.ly/3q9SZix', '20s_best_book.json', quiet=False)

Downloading...
From: https://bit.ly/3q9SZix
To: C:\Users\sizip\Python programming\20s_best_book.json
100%|██████████████████████████████████████████████████████████████████████████████| 92.9k/92.9k [00:00<00:00, 860kB/s]


'20s_best_book.json'

In [ ]:
# 2. json으로 DataFrame 생성
import pandas as pd

books_df = pd.read_json('20s_best_book.json')
print(books_df.shape)
books = books_df.loc[:, 'no':'isbn13']
books.head()

(200, 13)


,no,ranking,bookname,authors,publisher,publication_year,isbn13
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267


In [ ]:
# 3. 도서 한권에 대해 isbn으로 웹페이지 정보 추출
import requests

isbn = 9791190090018      # '우리가 빛의 속도로 갈 수 없다면'의 ISBN
url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'

r = requests.get(url.format(isbn))
print(r.text)







	<!DOCTYPE html >
	<html lang="ko">

<head>
	<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
	<meta http-equiv="Content-Type" content="text/html;charset=utf-8" />
	<meta http-equiv="Accept-CH" content="dpr, width, viewport-width, rtt, downlink, ect, UA, UA-Platform, UA-Arch, UA-Model, UA-Mobile, UA-Full-Version" />
	<meta http-equiv="Accept-CH-Lifetime" content="86400" />
	<meta name="referrer" content="unsafe-url" />
	<meta name="viewport" content="width=1170" />

	<title> 9791190090018 - 예스24</title> 

	<meta name="title" content="예스24" />
	<meta name="description" content="YOUR EVERY STORY 문화 콘텐츠 플랫폼, 예스24" />
	<meta name="keywords" content="인터넷 서점, 온라인 쇼핑, 상품 추천, 쇼핑몰, 상품 검색, 도서 정보, 국내도서, 외국도서, 전자책, eBook, 이북, 크레마, 공연, 콘서트, 뮤지컬, 음반, 예매, DVD, 블루레이, 예스24, YES24, 교보문고, 알라딘, 리센스, 예스24 도서용품, 친환경 PB 브랜드, 사은품, 굿즈" />


	<link title="YES24" rel="search" type="application/opensearchdescription+xml" href="//www.yes24.com/OpensearchDescription.xml">

	<script type="text/javascrip

In [ ]:
# 4. 웹페이지에서 href 정보를 추출하여 상세 페이지 가져오기
from bs4 import BeautifulSoup

soup = BeautifulSoup(r.text, 'html.parser')
prd_link = soup.find('a', attrs={'class':'gd_name'})

url = 'http://www.yes24.com'+prd_link['href']
print(url)

r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
print(prd_detail)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
from bs4 import BeautifulSoup
import requests # Import requests here as it's used in this block

soup = BeautifulSoup(r.text, 'html.parser')
prd_link = soup.find('a', attrs={'class':'gd_name'})

if prd_link is not None :
    url = 'http://www.yes24.com'+prd_link['href']
    print(url)

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
    print(prd_detail)

else:
    print("Could not find the product link on the page.")

Could not find the product link on the page.


In [ ]:
# 5 상세페이지에서 tr tag에서 텍스트 추출
prd_tr_list = prd_detail.find_all('tr')
# print(prd_tr_list)

for tr in prd_tr_list:
    if tr.find('th').get_text() == '쪽수, 무게, 크기':
        page_td = tr.find('td').get_text()
        break

print(page_td)

# 크기 데이터 : 4 번째
print(page_td.split()[4])

344쪽 | 496g | 130*198*30mm
130*198*30mm


In [ ]:
# 6. 전체 도서의 크기 구하기

# isbn으로 크기 정보 추출
def get_page_size(isbn):
    # Yes24 도서 검색 페이지 URL
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    # URL에 ISBN을 넣어 HTML 가져옵니다.
    r = requests.get(url.format(isbn))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    # 검색 결과에서 해당 도서를 선택합니다.
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return ''
    # 도서 상세 페이지를 가져옵니다.
    url = 'http://www.yes24.com'+prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    # 상품 상세정보 div를 선택합니다.
    prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
    # 테이블에 있는 tr 태그를 가져옵니다.
    prd_tr_list = prd_detail.find_all('tr')
    # 쪽수가 들어 있는 th를 찾아 td에 담긴 값을 반환합니다.
    for tr in prd_tr_list:
        if tr.find('th').get_text() == '쪽수, 무게, 크기':
            return tr.find('td').get_text().split()[4]
    return ''

# row에서 isbn13을 추출하여 get_page_size 실행
def get_page_size2(row):
    isbn = row['isbn13']
    return get_page_size(isbn)


In [ ]:
# 10개 도서에 대한 크기 정보 추출
top10_books = books.head(10)
page_size = top10_books.apply(get_page_size2, axis=1)
page_size.name = 'page_size'
print(page_size)

0    130*198*30mm
1    134*200*20mm
2    130*195*15mm
3    133*200*23mm
4    134*195*20mm
5    145*210*30mm
6    137*197*18mm
7                
8    140*210*16mm
9    145*210*20mm
Name: page_size, dtype: object


### (2) 크기와 함께 위에서 추출한 다른 정보를 동시에 저장하여 처리하는 코드를 작성

In [ ]:
top10_with_page_size = pd.merge(top10_books, page_size,
                                 left_index=True, right_index=True)
top10_with_page_size

,no,ranking,bookname,authors,publisher,publication_year,isbn13,page_size
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,130*198*30mm
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,134*200*20mm
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,130*195*15mm
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,133*200*23mm
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,134*195*20mm
5,6,6,피프티 피플 :정세랑 장편소설,지은이: 정세랑,창비,2016,9788936434243,145*210*30mm
6,7,7,목소리를 드릴게요 :정세랑 소설집,지은이: 정세랑,아작,2020,9791165300005,137*197*18mm
7,8,8,나미야 잡화점의 기적 :히가시노 게이고 장편소설,지은이: 히가시노 게이고 ;옮긴이: 양윤옥,현대문학,2012,9788972756194,
8,9,9,선량한 차별주의자,김지혜 지음,창비,2019,9788936477196,140*210*16mm
9,10,9,쇼코의 미소 :최은영 소설,지은이: 최은영,문학동네,2016,9788954641630,145*210*20mm
